This notebook explores both model index and opc ua scripts and contain examples of all the functions to make request to model index api and opc ua api servers.  

### Import Libraries

In [ ]:
# Import the required packeages
import pandas as pd
from typing import List, Dict
import sys
import os
import asyncio

### Import Scripts

In [ ]:
# Setting the path
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# Import model index functions
from model_index import ModelIndex

# Import OPC UA functions
from opc_ua import OPC_UA

In [ ]:
# Connection to the servers
model_index_url = "http://10.241.68.86:7001/v1/"
opcua_rest_url = "http://10.241.68.86:13371/"
opcua_server_url = "opc.tcp://10.241.80.4:4872"

# Model index API
model = ModelIndex(url=model_index_url)

# OPC UA API
tsdata = OPC_UA(rest_url=opcua_rest_url, opcua_url= opcua_server_url)

In [ ]:
# Input parameters for value data
# Parameters for aggregate historical data
start_time = "2022-07-05T07:55:14.544000Z" # 5th July aggregated data 
end_time = "2022-07-06T07:55:14.544000Z"
pro_interval = 600000 # 10 minutes processing time
agg_name = "Average"

### Download data from modelindex api

In [ ]:
# Listed sites on the model index api server
namespaces = model.get_namespace_array(return_format="dataframe")
namespaces

In [ ]:
# Types of Objects
obj_types = model.get_object_types(return_format="dataframe")
obj_types

In [ ]:
# Unique types of Objects
obj_types = model.get_object_types(return_format="dataframe")
obj_types_unique = obj_types[['Id', 'DisplayName', 'BrowseName']].drop_duplicates()
obj_types_unique

In [ ]:
# To get typeId by type name of an object
object_type_id = model.get_object_type_id_from_name("SiteType")
object_type_id

In [ ]:
# To get the objects of a type
obj_of_types = model.get_objects_of_type("SiteType", return_format="dataframe")
obj_of_types

In [ ]:
# Descendents of an object type
obj_descendents = model.get_object_descendants("StringSetType", obj_of_types, "PV_Assets", return_format="dataframe")
obj_descendents

In [ ]:
# All the sites on the OPC server
sites = model.get_objects_of_type('SiteType', return_format="dataframe")

# Selecting specific site 
sitename = 'BR-AP'
site = sites[sites['DisplayName'] == sitename]

In [ ]:
# Object descendants data of a specific site BR-AP
strings = model.get_object_descendants("StringSetType", site, "PV_Assets", return_format="dataframe")
strings

In [ ]:
# Ancestors of an object type
obj_ancestors = model.get_object_ancestors("TrackerType", strings, "PV_Serves", return_format="dataframe")
obj_ancestors 

### Download data from the opc ua api

In [ ]:
# Input parameters for value data
# For aggregate historical data
start_time = "2022-07-05T07:55:14.544000Z" # 5th July aggregated data 
end_time = "2022-07-06T07:55:14.544000Z"
pro_interval = 3600000 # 1 hour processing time
agg_name = "Average"

In [ ]:
# Live value data of trackers 
live_value = tsdata.get_live_values_data(['AngleMeasured', 'AngleSetpoint'], obj_ancestors)
live_value

In [ ]:
# 1 day aggregated historical trackers data
await tsdata.get_agg_hist_value_data(start_time, end_time, pro_interval, agg_name, obj_ancestors, ['AngleMeasured', 'AngleSetpoint'])

In [ ]:
# Reading downloaded tracker data 
tracker_df = pd.read_parquet('Data/data_chunk_0.parquet')
tracker_df